In [18]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()

from tqdm import tqdm_notebook, tqdm
from collections import deque

import warnings
warnings.filterwarnings('ignore')

In [19]:
class AI_Trader():
  
  def __init__(self, state_size, action_space=3, model_name="AITrader"): #Stay, Buy, Sell
    
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name = model_name
    
    # Define hyperparamaters
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
        
    # Call a function  to build a model trought this class constructor
    # More parameters could be ustilized to programaticaly define network size (layers and neurons)
    self.model = self.model_builder()
    
    
  def model_builder(self):
    metrics = [tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.FalseNegatives(), tf.keras.metrics.Accuracy()]
    model = tf.keras.models.Sequential()    
    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))    
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))    
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))    
    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))    
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=metrics)
    return model
  
  
  
  
  # Trade function that takes state as an input and returns an action 
  # to perform in perticular state 
  def trade(self, state):
    
    # Should we perform a renadom generated action or action defined in model?
    
    # If value from our random generator is smaller or equal to our epsilon 
    #     then we will retun a random action from action_space [0-3)
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    
    # If our random is greater than epsilon then we will use model to perform action
    actions = self.model.predict(state)
    # return only a one number defining an action (#Stay - 0 , Buy - 1, Sell - 2) 
    #    that has maximum probability
    return np.argmax(actions[0])
  
  
  
  def batch_train(self, batch_size):
    
    batch = []
    
    # Iterrate in momory, we do not want to randolmy select data as we are dealing with 
    #    time constraint data. We will always sample from the end of memory size of bath
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      # insert data from memory to batch      
      batch.append(self.memory[i])
    
    
    # Iterate trought batch of data and train the model for each sample from batch
    # Order of variables in for loop is important
    for state, action, reward, next_state, done in batch:
      # Reward if agent is in terminal state
      reward = reward
      # Check that agent is not in terminal state
      # If not in terminal state calculate reward for actions that could be played
      if not done:
        # Discounted total reward:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])        
      # Target variable that is predicted by the model (action)
      target = self.model.predict(state)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    # We will decrease epsilon parameter that is 1 as defined in __init__  so
    #    so we can stop performing random actions at some point
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

In [20]:
# Usually used at the end of a network for binary classifictation
# It changes range of input to scale of [0,1]
# So we can normalize input data for comparision day by day if they are on different scale
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

In [21]:
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

In [22]:
def state_creator(data, timestep, window_size):
  
  # starting day of our state
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    # Replicate member (data[0]) needed times
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
  state = []
  # Iterate trough whole windowed_data minus current state (-1)
  for i in range(window_size - 1):
    # Normalize the difference from current day and the next day
    # Because the prices can be very different and we want them on same scale
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  return np.array([state])

In [31]:
datos_historicos = yf.download('MSFT', start='2022-04-01', end='2023-10-14')
data = datos_historicos['Close']

[*********************100%***********************]  1 of 1 completed


In [24]:
data

Date
2022-04-01    309.420013
2022-04-04    314.970001
2022-04-05    310.880005
2022-04-06    299.500000
2022-04-07    301.369995
                 ...    
2023-10-09    329.820007
2023-10-10    328.390015
2023-10-11    332.420013
2023-10-12    331.160004
2023-10-13    327.730011
Name: Close, Length: 386, dtype: float64

In [25]:
window_size = 10

batch_size = 32
data_samples = len(data) - 1 # discard last value, that we will predict on

In [26]:
trader = AI_Trader(window_size)
trader.model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 32)                352       
                                                                 
 dense_5 (Dense)             (None, 64)                2112      
                                                                 
 dense_6 (Dense)             (None, 128)               8320      
                                                                 
 dense_7 (Dense)             (None, 3)                 387       
                                                                 
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


In [28]:
episode = 1
state = state_creator(data, timestep=0, window_size =window_size + 1)
total_profit = []
trader.inventory = []
df_historia = pd.DataFrame({'Fecha': [],
                            'Precio': [],
                            'Accion': [],
                            'Ganancia_total': []})

for t in tqdm(range(385)):
# First we will access action that is going to be taken by model 
    action = trader.trade(state)
    
    # Use action to get to next state(t+)
    next_state = state_creator(data=data, timestep=t+1, window_size=window_size + 1)
    # As we did not calculate anything up to this point reward is 0
    reward = 0
    
    if action == 1: #Buying
      # Put buyed stock to inventory to trade with
      trader.inventory.append(data[t])
      df_compra = pd.DataFrame({'Fecha': [data.index.values[t]],
                                  'Precio': [data[t]],
                                  'Accion': [action]})
      df_historia = pd.concat([df_historia, df_compra])
      print("AI Trader bought: ", stocks_price_format(data[t]))
      
    # To sell we need to have something in inventory  
    elif action == 2 and len(trader.inventory) > 0: #Selling
      # Check buy price, pop removes first value from list
      buy_price = trader.inventory.pop(0)
      
      # If we gain money (current price - buy price) we have reward 
      #    if we lost money then reward is 0
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      df_compra = pd.DataFrame({'Fecha': [data.index.values[t]],
                                  'Precio': data[t],
                                  'Accion': [action],
                                  'Ganancia_total': [stocks_price_format(data[t] - buy_price)]})
      df_historia = pd.concat([df_historia, df_compra])
      print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price))
      
    # if t is last sample in our dateset we are done
    #     we do not have any steps to perform in current episode
    if t == data_samples - 1:
      done = True
    else:
      done = False
    
    # Append all data to trader-agent memory, experience buffer
    trader.memory.append((state, action, reward, next_state, done))
    
    # change state to next state, so we are done with an episode
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")
    
    # Chekc if we have more information in our memory than batch size
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
  
  # Save the model every 10 episodes
    if episode % 10 == 0:
      trader.model.save("ai_trader_{}.h5".format(episode))

  0%|                                                   | 0/385 [00:00<?, ?it/s]

AI Trader bought:  $ 309.420013
1/1 [==============================] - 0s 12ms/step


  0%|                                           | 1/385 [00:02<13:55,  2.18s/it]

1/1 [==============================] - 0s 8ms/step


  1%|▏                                          | 2/385 [00:04<13:58,  2.19s/it]

...

 99%|████████████████████████████████████████▊| 383/385 [12:13<00:03,  1.88s/it]

1/1 [==============================] - 0s 7ms/step


100%|████████████████████████████████████████▉| 384/385 [12:15<00:01,  1.87s/it]

1/1 [==============================] - 0s 7ms/step
########################
TOTAL PROFIT: []
########################
1/1 [==============================] - 0s 7ms/step


100%|█████████████████████████████████████████| 385/385 [12:17<00:00,  1.92s/it]


In [30]:
df_historia.to_csv("modelo_con_cierre")

In [ ]:
datosDelModelo = {'Date': [data.index.values], 'Prices': [data.values], 'bought':[trader.inventory], 'sold':[buy_price], 'reward':[reward], 
 'profit': [total_profit]}

In [ ]:
df_model = pd.DataFrame()
for i in range(50):
    df_model['Date'] = data.index[i]

df_model

In [ ]:
total_profit

In [ ]:
df_historia  = pd.DataFrame({"fecha":[data.index.values[0]],
                        "precio":[data[0]],
                        "accion":[0]})
df_historia

In [ ]:
df_historia_app = pd.DataFrame({"fecha":[data.index.values[1]],
                        "precio":[data[1]],
                        "accion":[1]})

df_historia.append(df_historia_app)                


In [ ]:

# Importing pandas as pd
import pandas as pd
 
# creating the Series
series1 = pd.Series([1, 2, 3])
display('series1:', series1)
series2 = pd.Series(['A', 'B', 'C'])
display('series2:', series2)
 
# concatenating
display('After concatenating:')
display(pd.concat([series1, series2], 
                  axis = 1))

In [ ]:
for i in range(data):
    accion = np.random(0,2)
    if accion == 0:
        buy = dat
    


In [ ]:
df2 = pd.DataFrame()
df2['Date'] = data.index.values
df2['Prices'] = data.values
df2['Bought'] = trader.inventory
df2['Sold'] = buy_price
df2['Reward'] = reward
df2['Profift'] = total_profit

In [ ]:
for episode in range(1, episodes + 1):
  
  # To keep track of training process
  # .format populates {} with variables in .format(x,y)
  print("Episode: {}/{}".format(episode, episodes))
  
  # Create state
  # second parameter is timestep = 0
  state = state_creator(data, timestep=0, window_size =window_size + 1)
  
  total_profit = 0
  # Empty inventory before starting episode
  trader.inventory = []
  
  # One timestep is one day so number of timesteps we have represent data we have
  # tqdm is used for visualization
  for t in tqdm(range(data_samples)):
    
    # First we will access action that is going to be taken by model 
    action = trader.trade(state)
    
    # Use action to get to next state(t+)
    next_state = state_creator(data=data, timestep=t+1, window_size=window_size + 1)
    # As we did not calculate anything up to this point reward is 0
    reward = 0
    
    if action == 1: #Buying
      # Put buyed stock to inventory to trade with
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stocks_price_format(data[t]))
      
    # To sell we need to have something in inventory  
    elif action == 2 and len(trader.inventory) > 0: #Selling
      # Check buy price, pop removes first value from list
      buy_price = trader.inventory.pop(0)
      
      # If we gain money (current price - buy price) we have reward 
      #    if we lost money then reward is 0
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price) )
      
    # if t is last sample in our dateset we are done
    #     we do not have any steps to perform in current episode
    if t == data_samples - 1:
      done = True
    else:
      done = False
    
    # Append all data to trader-agent memory, experience buffer
    trader.memory.append((state, action, reward, next_state, done))
    
    # change state to next state, so we are done with an episode
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")
    
    # Chekc if we have more information in our memory than batch size
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
  
  # Save the model every 10 episodes
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))

In [ ]:
trader.memory